In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import backend, layers, models, utils, losses
from tensorflow.keras import datasets, Sequential, preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers

#from models.resnet import ResNet18
# Inside my model training code
from data_utils import CIFAR10Data

In [2]:
cifar10_data = CIFAR10Data()
x_train, y_train, x_test, y_test = cifar10_data.get_data(subtract_mean=True)

CIFAR10 Training data shape: (50000, 32, 32, 3)
CIFAR10 Training label shape (50000, 1)
CIFAR10 Test data shape (10000, 32, 32, 3)
CIFAR10 Test label shape (10000, 1)


In [3]:
len(x_train)

50000

In [4]:
# Inside my model training code
import wandb
from wandb.keras import WandbCallback

In [5]:
print(tf.__version__)

1.15.2


In [6]:
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [7]:
wandb.init(project="Active Learning")

wandb.config.embedding_size = 128

wandb.config.margin = 1.0
wandb.config.reduction_in_loss = 'mean' # 'none'

wandb.config.w_classif_loss = 1.0
wandb.config.w_loss_loss = 0

wandb.config.batch_size = 128

wandb.config.epoch = 50
wandb.config.lr = 0.1
wandb.config.milestones = [25, 35]
#wandb.config.epochl = 40
wandb.config.momentum = 0.9
wandb.config.wdecay = 5e-4


wandb: Currently logged in as: afospinat (use `wandb login --relogin` to force relogin)
/usr/local/lib/python3.6/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [8]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

In [25]:
wandb.config.input_shape = x_train.shape[1:]
wandb.config.classes_data = len(np.unique(y_train))
wandb.config.NUM_TRAIN = len(x_train) # N

wandb.config.ADDENDUM = 1000

In [37]:
##
#  Update the labeled dataset via loss prediction-based uncertainty measurement

# Randomly sample 10000 unlabeled data points
random.shuffle(unlabeled_set)
subset = unlabeled_set[:-1]

len(subset)

48999

In [26]:


'''''
##
# Learning Loss for Active Learning
NUM_TRAIN = 50000 # N
NUM_VAL   = 50000 - NUM_TRAIN
BATCH     = 128 # B
SUBSET    = 10000 # M
ADDENDUM  = 1000 # K

MARGIN = 1.0 # xi
WEIGHT = 1.0 # lambda

TRIALS = 3
CYCLES = 10

EPOCH = 200
LR = 0.1
MILESTONES = [160]
EPOCHL = 120 # After 120 epochs, stop the gradient from the loss prediction module propagated to the target model

MOMENTUM = 0.9
WDECAY = 5e-4
'''''

"''\n##\n# Learning Loss for Active Learning\nNUM_TRAIN = 50000 # N\nNUM_VAL   = 50000 - NUM_TRAIN\nBATCH     = 128 # B\nSUBSET    = 10000 # M\nADDENDUM  = 1000 # K\n\nMARGIN = 1.0 # xi\nWEIGHT = 1.0 # lambda\n\nTRIALS = 3\nCYCLES = 10\n\nEPOCH = 200\nLR = 0.1\nMILESTONES = [160]\nEPOCHL = 120 # After 120 epochs, stop the gradient from the loss prediction module propagated to the target model\n\nMOMENTUM = 0.9\nWDECAY = 5e-4\n"

In [27]:
import random

In [28]:
indices = list(range(wandb.config.NUM_TRAIN))
random.shuffle(indices)
labeled_set = indices[:wandb.config.ADDENDUM]
unlabeled_set = indices[wandb.config.ADDENDUM:]

In [30]:
len(labeled_set)

1000

In [31]:
len(unlabeled_set)

49000

In [ ]:
# Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.
indices = list(range(wandb.config.NUM_TRAIN))
random.shuffle(indices)
labeled_set = indices[:wandb.config.ADDENDUM]
unlabeled_set = indices[wandb.config.ADDENDUM:]

train_gen = train_datagen.flow(x_train[labeled_set],
                               y_train[labeled_set],
                               batch_size=wandb.config.batch_size)


dataloaders  = {'train': train_loader, 'test': test_loader}


# generate the classifier
Classification_with_AL = generate_model(wandb.config)

# 
Classification_with_AL.compile( optimizer=optimizer,
                                loss=loss_dict,
                                loss_weights=weigths_dict,
                                metrics=metrics)

# Active learning cycles
for cycle in range(CYCLES):


    # 
    Classification_with_AL.fit_generator(mod_data_gen(train_gen), 
                                         epochs=wandb.config.EPOCH, 
                                         steps_per_epoch= len(train_gen), 
                                         validation_data=mod_data_gen(test_gen),
                                         validation_steps=len(test_gen),
                                         validation_freq=1,
                                         callbacks=callbacks)


In [33]:
x_train[labeled_set].shape

(1000, 32, 32, 3)

In [11]:

# Generate model
def generate_model(config):

    def get_embedding_nets():
        return Sequential([layers.GlobalAveragePooling2D(),layers.Dense(config.embedding_size),layers.Activation("relu")])

    def get_classifcation_net():
        return Sequential([layers.GlobalAveragePooling2D(),layers.Dense(config.classes_data),layers.Activation("relu")])

    def get_classifcation_net():
        return Sequential([layers.GlobalAveragePooling2D(),layers.Dense(config.classes_data),layers.Activation("relu")])
    
    # generate the rest of the model
    inputs = tf.keras.Input(shape=config.input_shape)
    # add backbone
    with tf.variable_scope("backbone"):
        backbone = ResNet18(input_shape=config.input_shape, weights='imagenet',include_top=False)
        x = backbone(inputs)
    # make normal classification
    with tf.variable_scope("classification"):
        classification = get_classifcation_net()(x[0])
        #classification = tf.identity(classification,'out_classification')


    with tf.variable_scope("loss_learning"):
        # generate embeddings for each other output
        embeddings_list =[]
        for out in x[1:]:
            embeddings_list.append(get_embedding_nets()(out))
        embedding = layers.Concatenate()(embeddings_list)
        #embedding = tf.identity(embedding,'out_embedding')
        out_loss = layers.Dense(1)(embedding)
        out_loss = layers.Concatenate()([classification,out_loss])
        #out_loss = tf.identity(out_loss,'out_loss_learning')

    classifier = models.Model(inputs, [classification,embedding,out_loss])
    
    return classifier

In [12]:
def compute_loss_loss_v0(c_true, c_pred, l_pred, margin=1.0, reduction='mean'):

    assert y_pred.shape[0] % 2 == 0, 'the batch size is not even.'
    l_pred_r = l_pred[::-1]
    assert l_pred.shape == l_pred_r.shape
    
    l_pred = (l_pred - l_pred_r)[:y_pred.shape[0]//2]
    
    # c_true is just the classification as the l_true is calculated here
    scc = tf.keras.losses.SparseCategoricalCrossentropy(reduction='none')
    class_loss = scc(c_true,c_pred)
    
    target = (class_loss - class_loss[::-1])[:class_loss.shape[0]//2]
    target = tf.stop_gradient(target)
    
    one = (2 * tf.math.sign(  tf.clip_by_value( target, 0, 1))) - 1
    
    if reduction == 'mean':
        loss = tf.reduce_sum(tf.clip_by_value(margin - one * l_pred, 0,10000))
        loss = loss / tf.cast(l_pred.shape[0], tf.float64)  # Note that the size of l_pred is already halved
    elif reduction == 'none':
        loss = tf.clip_by_value(margin - one * l_pred, 0,10000)
    else:
        NotImplementedError()
    
    return loss,target

In [ ]:
import tensorflow as tf

def compute_loss(y_true, y_pred):
    margin=1.0
    reduction= 'mean'

    assert y_pred.shape[0] % 2 == 0, 'the batch size is not even.'
    
    #assert y_pred.shape == y_pred.flip(0).shape     
    # classification prediction 
    c_pred = y_pred[:,:-1]
    # loss prediction
    l_pred = y_pred[:,-1]
    l_pred_r = l_pred[::-1]
    assert l_pred.shape == l_pred_r.shape
    
    l_pred = (l_pred - l_pred_r)[:y_pred.shape[0]//2]
    
    # y_true is just the classification as the l_true is calculated here
    scc = tf.keras.losses.SparseCategoricalCrossentropy(reduction='none')
    class_loss = scc(y_true,c_pred)
    
    target = (class_loss - class_loss[::-1])[:class_loss.shape[0]//2]
    target = tf.stop_gradient(target)
    
    one = (2 * tf.math.sign(  tf.clip_by_value( target, 0, 1))) - 1
    
    if reduction == 'mean':
        loss = tf.reduce_sum(tf.clip_by_value(margin - one * l_pred, 0,10000))
        loss = loss / tf.cast(l_pred.shape[0], tf.float64)  # Note that the size of l_pred is already halved
    elif reduction == 'none':
        loss = tf.clip_by_value(margin - one * l_pred, 0,10000)
    else:
        NotImplementedError()
    
    return loss




In [13]:
class Loss_leaning_loss(losses.Loss):
    def __init__(self, margin=1.0, reduction='mean', name="Learning_loss"):
        super().__init__(name=name)
        self.margin=1.0
        self.reduction= 'mean'


    def call(self, y_true, y_pred):  

        c_pred = y_pred[:,:-1]
        # loss prediction
        l_pred = y_pred[:,-1]
        l_pred_r = l_pred[::-1]
        #assert tf.shape(y_pred) == tf.shape(l_pred_r)

        l_pred = (l_pred - l_pred_r)[:y_pred.shape[0]//2]

        # y_true is just the classification as the l_true is calculated here
        scc = losses.SparseCategoricalCrossentropy(reduction='none')
        class_loss = scc(y_true,c_pred)

        l_true = (class_loss - class_loss[::-1])[:class_loss.shape[0]//2]
        l_true = tf.stop_gradient(l_true)

        one = (2 * tf.math.sign(  tf.clip_by_value( l_true, 0, 1))) - 1

        if self.reduction == 'mean':
            loss = tf.reduce_sum(tf.clip_by_value(self.margin - one * l_pred, 0,10000))
            loss = tf.math.divide(loss , tf.cast(tf.shape(l_pred)[0], loss.dtype) ) # Note that the size of l_pred is already halved
        elif self.reduction == 'none':
            loss = tf.clip_by_value(self.margin - one * l_pred, 0,10000)
        else:
            NotImplementedError()

        return loss

In [15]:
# generate the classifier
Classification_with_AL = generate_model(wandb.config)

NameError: name 'ResNet18' is not defined

In [16]:
Classification_with_AL.output_names

NameError: name 'Classification_with_AL' is not defined

In [17]:
# generate dataloader for train
train_datagen = ImageDataGenerator(
        width_shift_range=[-2,2],
        height_shift_range=[-2,2],
        horizontal_flip=True)

# generate dataloader for test
test_datagen = ImageDataGenerator()


# losses
loss_dict={Classification_with_AL.output_names[0]:tf.keras.losses.SparseCategoricalCrossentropy(),
           Classification_with_AL.output_names[2]:Loss_leaning_loss()}

# weigths for each loss
weigths_dict={ Classification_with_AL.output_names[0]:wandb.config.w_classif_loss,
               Classification_with_AL.output_names[1]:0, 
               Classification_with_AL.output_names[2]:wandb.config.w_loss_loss}

# metrics to compute
metrics={ Classification_with_AL.output_names[0]:tf.keras.metrics.SparseCategoricalAccuracy()}

# Optimizer
optimizer = tf.keras.optimizers.SGD( learning_rate=wandb.config.lr, momentum=wandb.config.momentum)

# callback to define a LearningRateScheduler
callbacks = []
# Change learning rate
callbacks.append( tf.keras.callbacks.LearningRateScheduler(scheduler) )
# get te values to wandb
callbacks.append( WandbCallback()   )
# save the best model
#callbacks.append( tf.keras.callbacks.ModelCheckpoint( filepath='model.{epoch:02d}-{val_loss:.2f}.h5'), save_freq='5', **kwargs  )

NameError: name 'Classification_with_AL' is not defined

In [ ]:
# generate dataloader for train
train_datagen = ImageDataGenerator(
        width_shift_range=[-2,2],
        height_shift_range=[-2,2],
        horizontal_flip=True)

# generate dataloader for test
test_datagen = ImageDataGenerator()

def scheduler(epoch):
    lr= wandb.config.lr
    for i in wandb.config.milestones:
        if epoch>i:
            lr*=0.1
    return lr

# get the data to test 
test_gen = test_datagen.flow(x_test,y_test,batch_size=wandb.config.batch_size,shuffle=False)
train_gen = train_datagen.flow(x_train,y_train,batch_size=wandb.config.batch_size)

# callback to define a LearningRateScheduler
callbacks = []
# Change learning rate
callbacks.append( tf.keras.callbacks.LearningRateScheduler(scheduler) )
# get te values to wandb
callbacks.append( WandbCallback()   )


def mod_data_gen(generator):
    while True:
        X,Y = generator.next()
        yield X, [Y, Y]

In [ ]:
Classification_with_AL.compile(  optimizer=optimizer,
                                loss=loss_dict,
                                loss_weights=weigths_dict,
                                metrics=metrics)

In [ ]:
Classification_with_AL.fit_generator(mod_data_gen(train_gen), 
                                     epochs=wandb.config.epoch, 
                                     steps_per_epoch= len(train_gen), 
                                     validation_data=mod_data_gen(test_gen),
                                     validation_steps=len(test_gen),
                                     callbacks=callbacks)
